In [ ]:
!mc cp s3/$VAULT_TOP_DIR/Accords/Construction_dataset_public/Dataset_public_accords_teletravail_Dares.parquet .

In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document, Generation, LLMResult
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import requests
from nltk.tokenize import word_tokenize
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

!python -m spacy download fr_core_news_lg

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)



model_kwargs = {'device': 'cuda'}
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs,show_progress=False)

In [ ]:
file="Dataset_public_accords_teletravail_Dares.parquet"
df=pd.read_parquet(file)

In [ ]:


nlp = spacy.load("fr_core_news_lg")
pipe = nlp.pipe(df.texte_complet_accord, n_process=5,
                disable=["tagger", "parser", "attribute_ruler", "ner"])

def preprocess_token(token):
    if not (token.is_stop or token.is_punct) and token.is_alpha and len(token) >= 3:
        return token.lemma_
    else:
        return ""

def preprocess_text(text_nlp):
    text_pretraite_list = [preprocess_token(token) for token in text_nlp if token]
    text_pretraite = " ".join(text_pretraite_list)
    return text_pretraite

df["data_pretraites"] = [preprocess_text(texte) for texte in pipe]

In [ ]:
t=df.data_pretraites[0]

vectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer.fit([t])
vocab = vectorizer.get_feature_names_out()

comptages = np.array(vectorizer.transform([t]).sum(0))[0]
comptages_voc = list(zip(vocab, comptages))
comptages_voc = sorted(comptages_voc, key=lambda x: x[1], reverse=True)
comptages_voc = pd.DataFrame(comptages_voc, columns=['mot', 'frequence'])  

comptages_voc.head(5)

In [ ]:
#def top5(t):
    

In [ ]:
vector_store = Chroma()
for index, row in tqdm(df.iterrows(), total=len(df)):
    print(index)
    text=df.texte_complet_accord[index]
    texts = text_splitter.create_documents([text])
    for i,t in enumerate(texts):
        embedding=embedder.encode(t)
        vector_store.add_documents({"id": f"{index}_{i}", "vector": embedding.tolist(), "metadata": {"text": f"{index} : {top5(t)}"})
        del(embedding)

In [ ]:
vector_store.similarity_search("télétravail",2)

 